In [1]:
def fetch_missing_wikipedia_articles(url):
    photos = json.loads(requests.get(url).text)
    temp_results = []
    for obs in photos["results"]:
        if len(obs["taxon"]["name"].split(" "))==2:
            if obs["taxon"]["wikipedia_url"] is None:
                result = dict()
                result["inat_obs_id"] = obs["id"]
                result["inat_taxon_id"] =  obs["taxon"]["id"]
                result["taxon_name"] = obs["taxon"]["name"]
                temp_results.append(result)
    to_verify = []
    for temp in temp_results:
        if temp["taxon_name"] not in to_verify:
            to_verify.append(temp["taxon_name"])
    verified = verify_wikidata(to_verify)
    results = []
    for temp in temp_results:
        if temp["taxon_name"] in verified:
            results.append(temp)
    return results

def verify_wikidata(taxon_names):
    progress = widgets.IntProgress(
                value=1,
                min=0,
                max=len(taxon_names)/50,
                description='Wikidata:',
                bar_style='', # 'success', 'info', 'warning', 'danger' or ''
                style={'bar_color': 'blue'},
                orientation='horizontal')
    display(progress)
    verified = []
    i = 1
    for chunks in [taxon_names[i:i + 50] for i in range(0, len(taxon_names), 50)]:
        names=" ".join('"{0}"'.format(w) for w in chunks)
        query = f"""
             SELECT DISTINCT ?taxon_name (COUNT(?item) AS ?item_count) (COUNT(?article) AS ?article_count)   WHERE {{
                        VALUES ?taxon_name {{{names}}}
                {{?item wdt:P225 ?taxon_name .}}
               UNION
               {{?item wdt:P225 ?taxon_name .
                ?article schema:about ?item ;
                         schema:isPartOf 	<{os.environ["wikipedia"]}> .}}
                 UNION
               {{?basionym wdt:P566 ?item ;
                          wdt:P225 ?taxon_name .
               ?article schema:about ?item ;
                        schema:isPartOf 	<{os.environ["wikipedia"]}> .}}
               UNION
               {{
                  ?item wdt:P225 ?taxon_name .
                  ?wikidata_item wdt:P460 ?item ;
                                 schema:isPartOf 	<{os.environ["wikipedia"]}> .
               }}
               UNION
               {{?basionym wdt:P566 ?item .
                ?item wdt:P225 ?taxon_name .
               ?article schema:about ?basionym ;
                        schema:isPartOf 	<{os.environ["wikipedia"]}> .}}
      }} GROUP BY ?taxon_name
            """

        url = "https://query.wikidata.org/sparql?format=json&query="+query
        #print(url)
        progress.value = i
        i+=1
        try:
            results = json.loads(requests.get(url).text)
        except:
            continue
        for result in results["results"]["bindings"]:
            if result["article_count"]["value"]=='0':
                verified.append(result["taxon_name"]["value"])
    return verified
